In [2]:
import pandas as pd

df_moma = pd.read_csv("../data/raw/artworks.csv")
df_moma.head(5)

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,OnView,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
0,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(male),1896,Ink and cut-and-pasted painted pages on paper,...,NaN,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN,NaN
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(male),1987,Paint and colored pencil on print,...,NaN,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN,NaN
2,"Villa project, outside Vienna, Austria (Elevat...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(male),1903,"Graphite, pen, color pencil, ink, and gouache ...",...,NaN,NaN,NaN,NaN,34.3000,NaN,NaN,31.8000,NaN,NaN
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,"(French and Swiss, born Switzerland 1944)",(),(1944),(0),(male),1980,Photographic reproduction with colored synthet...,...,NaN,NaN,NaN,NaN,50.8000,NaN,NaN,50.8000,NaN,NaN
4,"Villa project, outside Vienna, Austria (Exteri...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(male),1903,"Graphite, color pencil, ink, and gouache on tr...",...,NaN,NaN,NaN,NaN,38.4000,NaN,NaN,19.1000,NaN,NaN


I will start by removing unnecessary columns.

In [4]:
df_moma = df_moma.drop(columns=['ConstituentID','ArtistBio','Nationality','BeginDate','EndDate','Date','Medium','Dimensions','CreditLine','AccessionNumber','Classification','Department','Cataloged','ObjectID','URL','ImageURL','OnView','Circumference (cm)','Depth (cm)','Diameter (cm)','Height (cm)','Length (cm)','Weight (kg)','Width (cm)','Seat Height (cm)','Duration (sec.)'])
     

In [5]:
df_moma.head(5)

,Title,Artist,Gender,DateAcquired
0,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,(male),1996-04-09
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,(male),1995-01-17
2,"Villa project, outside Vienna, Austria (Elevat...",Emil Hoppe,(male),1997-01-15
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,(male),1995-01-17
4,"Villa project, outside Vienna, Austria (Exteri...",Emil Hoppe,(male),1997-01-15


Next, I will rename the columns.

In [6]:
df_moma = df_moma.rename(columns={'Title':'title','Artist':'artist','Gender':'gender','DateAcquired':'date_acquired'})
df_moma.head(5)

,title,artist,gender,date_acquired
0,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,(male),1996-04-09
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,(male),1995-01-17
2,"Villa project, outside Vienna, Austria (Elevat...",Emil Hoppe,(male),1997-01-15
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,(male),1995-01-17
4,"Villa project, outside Vienna, Austria (Exteri...",Emil Hoppe,(male),1997-01-15


Next, I will change title, artist, and gender to string data types.

In [12]:
df_moma.title = df_moma.title.astype('string')
df_moma.artist = df_moma.artist.astype('string')
df_moma.gender = df_moma.gender.astype('string')


In [13]:
df_moma.dtypes

title            string[python]
artist           string[python]
gender           string[python]
date_acquired            object
dtype: object

Next, I will change the "date" column to a datetime data type.

In [15]:
df_moma['date_acquired'] = pd.to_datetime(df_moma['date_acquired'])

In [16]:
df_moma.dtypes

title            string[python]
artist           string[python]
gender           string[python]
date_acquired    datetime64[ns]
dtype: object